# Combine into single dataset 
    A) Satelite data from Google Earth engine 
    B) desinventar events 
    C) set of drought events found by searching for news articles 

Every row in the dataframe will correspond to a particular month/year + region. 
For every drought event, we add a "TRUE" to the collumn corresponding to the reported date. 
Also have the option to extend this event by, for example, ±1 month

In [37]:
%matplotlib inline 
import pandas as pd 
import matplotlib.pylab as plt 
import numpy as np 
from datetime import date

# Satelite data from Google Earth engine

for general description:
https://developers.google.com/earth-engine/datasets

The datasets/ collumn names we use for now: 
* NDVI: 'Normalized Difference Vegitation Index" (scale by 0.0001)
* EVI: 'Enhanced Vegitation Index' (scale by 0.0001)
* precipitation: rainfall measured in mm/hrs (summed over a month to get mm/month)
* hourlyPrecipRate: (also?) rainfall measured in mm/hrs (summed over month to get mm/month)
* LST_Day_1km: Land surface temperature during daytime in  50 Kelvins (multiply number by 0.02 to get Kelvins)
* LST_Night_1km: Land surface temperature during nighttime in  50 Kelvins (multiply number by 0.02 to get Kelvins)
* Evap_tavg: Evapotranspiration measured in kg m^{-2} s^{-1}  ("flux per second") 
* Rainf_f_tavg: Total precipitation rate measured in kg m^{-2} s^{-1}  ("flux per second") 
* SoilMoi00_10cm_tavg: Soil moisture (0 - 10 cm underground)    in m^3 m-3
* SoilMoi10_40cm_tavg: Soil moisture (10 - 40 cm underground)   in m^3 m-3
* SoilMoi40_100cm_tavg: Soil moisture (40 - 100 cm underground) in m^3 m-3
* SoilMoi100_200cm_tavg: Soil moisture (100 - 200 cm underground) in m^3 m-3
* SoilTemp00_10cm_tavg: Soil temperature (0 - 10 cm underground) in K 
* SoilTemp0_40cm_tavg: Soil temperature (10 - 40 cm underground) in K 
* SoilTemp40_100cm_tavg: Soil temperature (40 - 100 cm underground) in K 
* SoilTemp100_200cm_tavg:Soil temperature (100 - 200 cm underground) in K 
* Tair_f_tavg: Near surface air temperature in Kelvin
* Wind_f_tavg: Near surface wind speed in m/s 

using some built-in R-code we further determined 
* SPEI: A measure of precipitation minus evapotranspiration. We have these averaged over the last 1, 2, 3, ...,12 months


we data have two countries: Uganda and Kenya (admin level 1 refers to a district, Red Cross terminology) 

In [38]:
UG_satelite = pd.read_csv('UG_merged_adm1_with_spei.csv')
KE_satelite = pd.read_csv('KE_merged_adm1_with_spei.csv')

### prepare the date column 
We are working with monthly averages, so we just use the 1st of the month as a placeholder. 
Although event data is available for particular publication days, I here do not use that information (see below when combining), as this is not an accurate indicator of the actual drought event anyways.  

The following is simply in order to visualize time series later on. 

In [39]:
def get_date(row):
    try:
        year = row['Year']
        month= row['Month']
        day = row['Day']
    except:
        year = row['year']
        month= row['month']
        day = row['day']
    
    try:
        date_of_event = date(year, month, day)
    except:
        # --- if we do not have the day, that is OK for drought, 
        # --- just put it on the 1st of the month --- 
        if (year!=0 and month!=0):
            date_of_event = date(year,month,1)
        else:
            date_of_event = np.nan
    return date_of_event

UG_satelite['Day'] = 1   # As we are working with 
UG_satelite['Date'] = UG_satelite.apply(get_date, axis=1)

KE_satelite['Day'] = 1   # As we are working with 
KE_satelite['Date'] = KE_satelite.apply(get_date, axis=1)

### rename collumns to have (slightly) more intuitive names 

In [40]:
columns_to_keep_satelite = { 'ADM0_EN': 'Country',
                    'ADM1_EN': 'District',
                    'Year':'year',
                    'Month':'month',
                    'Day':'day',
                    'Date':'date',
                    'NDVI_mean':'NDVI',
                    'EVI_mean': 'EVI',
                    'precipitation_sum':'precipitation_per_hour_v1',
                    'hourlyPrecipRate_sum':'precipitation_per_hour_v2',
                   'LST_Day_1km_mean':'surface_temperature_daytime',
                   'LST_Night_1km_mean':'surface_temperature_nighttime',
                    'Evap_tavg_mean': 'evapotranspiration',
                    'Rainf_f_tavg_mean':'rainfall',
                    'SoilMoi00_10cm_tavg_mean':'SoilMoisture00_10cm',
                    'SoilMoi10_40cm_tavg_mean':'SoilMoisture10_40cm',
                    'SoilMoi40_100cm_tavg_mean':'SoilMoisture40_100cm',
                    'SoilMoi100_200cm_tavg_mean':'SoilMoisture100_200cm',
                    'SoilTemp00_10cm_tavg_mean':'SoilTemperature00_10cm',
                    'SoilTemp10_40cm_tavg_mean':'SoilTemperature10_40cm',
                    'SoilTemp40_100cm_tavg_mean':'SoilTemperature40_100cm',
                    'SoilTemp100_200cm_tavg_mean':'SoilTemperature100_200cm',
                    'Tair_f_tavg_mean': 'air_temperature',
                    'Wind_f_tavg_mean': 'wind_speed',
                    'spei_1':'SPEI_1month',
                    'spei_2':'SPEI_2month',
                    'spei_3':'SPEI_3month',
                    'spei_4':'SPEI_4month',
                    'spei_5':'SPEI_5month',
                    'spei_6':'SPEI_6month',
                    'spei_7':'SPEI_7month',
                    'spei_8':'SPEI_8month',
                    'spei_9':'SPEI_9month',
                    'spei_10':'SPEI_10month',
                    'spei_11':'SPEI_11month',
                    'spei_12':'SPEI_12month'
                   }




UG = UG_satelite[columns_to_keep_satelite.keys()]
UG.rename(columns=columns_to_keep_satelite, inplace=True)

KE = KE_satelite[columns_to_keep_satelite.keys()]
KE.rename(columns=columns_to_keep_satelite, inplace=True)


### Combine the countries into a single dataframe/file

In [41]:
DroughtData = pd.concat([UG,KE])
DroughtData.reset_index(inplace=True)
DroughtData.drop('index',axis=1, inplace=True)

### transform to proper units 

In [42]:
DroughtData['NDVI'] *= 0.0001
DroughtData['EVI'] *= 0.0001
DroughtData['surface_temperature_daytime'] *= 0.02
DroughtData['surface_temperature_nighttime'] *= 0.02

In [43]:
SPEI_data = pd.read_csv('UG_merged_adm1_with_spei.csv')
SPEI_data

,ADM0_EN,ADM0_PCODE,ADM1_EN,ADM1_PCODE,DayOfYear_mean,DetailedQA_mean,EVI_mean,Month,NDVI_mean,OBJECTID,...,spei_3,spei_4,spei_5,spei_6,spei_7,spei_8,spei_9,spei_10,spei_11,spei_12
0,Uganda,UG,ABIM,UG314,76.344181,2112.418701,1371.556048,3,2707.118608,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uganda,UG,ABIM,UG314,110.343445,2301.095170,1846.158551,4,3684.379945,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Uganda,UG,ABIM,UG314,143.201514,2112.012438,3653.801670,5,6247.636389,1,...,0.075238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Uganda,UG,ABIM,UG314,174.596601,2118.293901,4168.914213,6,6698.680552,1,...,0.917779,0.711107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uganda,UG,ABIM,UG314,206.523138,2124.983729,4584.023762,7,7177.464449,1,...,1.070958,0.836407,0.635385,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28531,Uganda,UG,BUKEDEA,UG219,113.316940,2153.906148,2680.886644,4,4598.645615,16,...,0.009296,-0.081664,0.113909,-0.515754,-0.570638,-1.382297,-1.596803,-1.614692,-1.218336,-0.635311
28532,Uganda,UG,BUKEDEA,UG219,141.007488,3391.122871,3714.824144,5,6209.868106,16,...,0.412603,0.418563,0.415291,0.488249,-0.057029,-0.262793,-1.051117,-1.350437,-1.454127,-0.998408
28533,Uganda,UG,BUKEDEA,UG219,175.612931,2225.823801,4311.129146,6,7060.787680,16,...,0.862155,0.846395,0.783062,0.791293,0.808201,0.376809,0.135418,-0.588563,-0.835667,-0.936488
28534,Uganda,UG,BUKEDEA,UG219,206.123682,2147.397208,4368.599164,7,6934.296439,16,...,1.381775,1.161586,1.231014,1.126487,1.111064,1.081678,0.674679,0.430766,-0.331863,-0.604257


# Droughts reported in the news 

In [44]:
import pandas as pd
droughts_from_news = pd.read_csv('news_droughts_Uganda_districts.csv',index_col=0)
droughts_from_news.reset_index(inplace=True)
droughts_from_news.drop('index',inplace=True,axis=1)

In [45]:
droughts_from_news.head()

,date,subregion,district,county,subcounty,sentence,article_title,article_url
0,2017-09-02,NaN,AMUDAT,POKOT,NaN,"These include Baringo, Garissa, Isiolo, Marsab...","Kenya experiencing one of the worst drought, d...",https://capitalradio.co.ug/kenya-experiencing-...
1,2017-07-09,NaN,"NTUNGAMO, BUSHENYI, MARACHA, IGANGA, NEBBI, NA...","ISINGIRO, MARACHA","NAMUTUMBA, NTUNGAMO, NEBBI, KIRYANDONGO","They include Arua, Maracha, Nebbi, Yumbe, Igan...","Ugandans are faced with food insecurity, offic...",https://capitalradio.co.ug/ugandans-faced-food...
2,2017-04-24,NaN,GULU,NaN,NaN,Persistent drought dries up Dam in Gulu.,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...
3,2017-04-24,NaN,GULU,NaN,NaN,Persistent drought dries up Dam in Gulu,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...
4,2017-04-24,NaN,GULU,NaN,NaN,Two large dams supplying water to Gulu town ha...,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...


### complete dataframe (missing values)
Some of the events do not have a reported district, but do still have lower/higher level regions specified. Sometimes the quote ('sentence' collumn) has a list of districts. 

For now, I ignored the events without explicit mentioning of the district. Should probably complete dataframe later. 


### prepare merge with satelite data
Make collumns with the month + year (+ day). When comparing to satelite data, we can then check for the district + year + month.  


In [46]:
def get_day_month_year(date_str):
    date = date_str.split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    return day, month, year



days = []
months = []
years  = []

for i in range(len(droughts_from_news)):
    day, month, year = get_day_month_year(droughts_from_news['date'][i])
    days.append(day)
    
    if month[0] == '0':
        months.append(month[1])
    else: 
        months.append(month)
    years.append(year)

droughts_from_news['year'] = years
droughts_from_news['month'] = months
droughts_from_news['day'] = days 

### loop over all reported events (with a district name, see above) and manually merge with satelite data
As a single district can have multiple events reported and a single event can apply to several districts, I decided to make a loop to merge it with satelite data. 



In [47]:
droughts_from_news = droughts_from_news[droughts_from_news['district'].notnull()]


DroughtData['drought_reported'] = False
DroughtData['drought_news_article'] = False

# --- loop over events --- 
for i in droughts_from_news.index:
    # --- extract the districts belonging to the reported event ---- 
    districts_effected = droughts_from_news['district'][i].split(',')
    
    # --- filter the satelite data by the reported date + district ---- 
    for dist in districts_effected:
        subset = DroughtData[(DroughtData['District']==dist )& 
                             (DroughtData['month'] == int(droughts_from_news.loc[i]['month']))&
                             (DroughtData['year'] == int(droughts_from_news.loc[i]['year']))]
        
        
        # --- record that there has been an event ---- 
        for j in subset.index:
            DroughtData['drought_reported'].loc[j] = True
        # --- record the event comes from news articles --- 
            DroughtData['drought_news_article'].loc[j] = True

        
print(len(DroughtData[DroughtData['drought_reported']]), len(DroughtData[DroughtData['drought_news_article']]))

81 81


# Desinventar data 

similar as above. Just keep track of which events where in desinventar

In [48]:
UG_desinventar = pd.read_csv('DI_export_uga.csv')
UG_droughts = UG_desinventar[UG_desinventar['event']=="DROUGHT"]

KE_desinventar = pd.read_csv('DI_export_ken.csv')
KE_droughts = KE_desinventar[KE_desinventar['event']=="DROUGHT"]

# --- actual day of the event, only use if we have year + month ---  
UG_droughts['date_of_event'] = KE_droughts.apply(get_date,axis=1)
KE_droughts['date_of_event'] = KE_droughts.apply(get_date,axis=1)


# --- overwrite the day to be the 1st of the month in order to do the merging
UG_droughts['day'] = 1 
UG_droughts['date'] = UG_droughts.apply(get_date,axis=1)

KE_droughts['day'] = 1 
KE_droughts['date'] = UG_droughts.apply(get_date,axis=1)

# --- for merge --- 
UG_droughts['Country'] = 'Uganda'
KE_droughts['Country'] = 'Kenya'

columns_to_keep_desinventar = {'Country':'Country',
                              'admin_level_0_name':'District',
                              'year':'year',
                              'month':'month',
                              'day':'day',
                              'date':'date',
                              'date_of_event':'date_of_event',
                              'latitude':'latitude',
                              'longitude':'longitude'}

UG_droughts_RK = UG_droughts[columns_to_keep_desinventar.keys()]
UG_droughts_RK.rename(columns=columns_to_keep_desinventar, inplace=True);

KE_droughts_RK = KE_droughts[columns_to_keep_desinventar.keys()]
KE_droughts_RK.rename(columns=columns_to_keep_desinventar, inplace=True);

C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

### Combine the countries into a single dataframe/file

In [49]:
desinventar = pd.concat([UG_droughts_RK,KE_droughts_RK])
desinventar.reset_index(inplace=True)
desinventar.drop('index',axis=1, inplace=True)

## no reported districts 
same as done for the news articles. Most probably at another level than 'admin level 1' the region might still be indicated in desinventar. 

In [50]:
desinventar = desinventar[desinventar['District'].notnull()]
print(len(desinventar))

1229


## properly combine this with the satelite data 
similar as done above for events from news articles 

In [51]:
# DroughtData['drought_reported'] = False
DroughtData['drought_desinventar'] = False
print(len(DroughtData[DroughtData['drought_reported']]), len(DroughtData[DroughtData['drought_news_article']]))

# --- loop over events --- 
for i in desinventar.index:
    # --- extract the districts belonging to the reported event ---- 
    districts_effected = desinventar['District'][i].upper()
    
    # --- filter the satelite data by the reported date + district ---- 
    subset = DroughtData[(DroughtData['District'].str.upper()==districts_effected )& 
                             (DroughtData['month'] == int(desinventar.loc[i]['month']))&
                             (DroughtData['year'] == int(desinventar.loc[i]['year']))]

    # --- record that there has been an event ---- 
    for j in subset.index:
        DroughtData.at[j, 'drought_reported'] = True
    # --- record the event comes from news articles --- 
        DroughtData.at[j, 'drought_desinventar'] = True

print(len(DroughtData[DroughtData['drought_reported']]),
      len(DroughtData[DroughtData['drought_news_article']]),
      len(DroughtData[DroughtData['drought_desinventar']]))

81 81
361 81 282


## save to a csv file

In [52]:
DroughtData.to_csv('Droughts_satelite_and_events.csv', index=False)

# Loosen the date of the event we record in the dataset 
Droughts obviously last longer than a day, also longer than a month. 
Therefore, also add 'TRUE' to the appropriate collumns at the reported date ± $x$ months. 

for now, I used 1 month 

### code snippet to produce combinations of years+months that we want to associate with an event 

In [16]:
# margin_of_error_months = 1
# month_original =1 
# year_original = 2016 


def get_entries_for_event(month_original, year_original, margin_of_error_months):
    months_of_year = range(1,13)
    entries_current_year = list( zip( months_of_year, [year_original]*len(months_of_year)   ) )
    entries_previous_year = list( zip( months_of_year, [year_original-1]*len(months_of_year)   ))
    entries_next_year = list( zip( months_of_year, [year_original+1]*len(months_of_year)   ) )


    # --- within current year ---
    if ((month_original + margin_of_error_months) <= 12) and ( (month_original - margin_of_error_months >= 1) ):

        start_event = month_original - margin_of_error_months
        stop_event = month_original + margin_of_error_months  


        adjust_these_entries = entries_current_year[(start_event-1):(stop_event ) ]


    # --- event started previous year --- 
    elif (month_original - margin_of_error_months < 1):


        # --- take the remaining months from last year ----
        start_event = month_original - margin_of_error_months
        stop_event = 12 

        adjust_these_entries = entries_previous_year[(start_event-1):(stop_event ) ] 


        # --- future months should fit within current year --- 
        start_event = 1  #January
        stop_event = month_original + margin_of_error_months
        adjust_these_entries += entries_current_year[(start_event-1):(stop_event ) ] 


    # --- event ends next year --- 
    elif (month_original + margin_of_error_months >12):
        # --- previous months come from current year ---- 
        start_event = month_original - margin_of_error_months
        stop_event = 12
        adjust_these_entries = entries_current_year[(start_event-1):(stop_event ) ] 


        # --- take remaining months from next year ---- 
        start_event = 1 
        stop_event = month_original + margin_of_error_months - 12 
        adjust_these_entries += entries_next_year[(start_event-1):(stop_event ) ] 
    
    
    
    return adjust_these_entries



# adjust_these_entries = get_entries_for_event(month_original, year_original, margin_of_error_months)
    

# print adjust_these_entries

### need a copy of the original dataset. Run this cell just once 

In [17]:
OriginalData = DroughtData.copy()

### adjust dataset to include 'exented' events 

In [18]:
margin_of_error_months = 1

original_events = OriginalData[OriginalData['drought_reported']].copy()

counter = 1
for i in original_events.index:
    month_original = original_events.loc[i]['month']
    year_original  = original_events.loc[i]['year']
    
    
    
    # --- filter the satelite data by the reported district ---- 
    district_data = OriginalData[(OriginalData['District']==original_events.loc[i]['District'] )]

    # --- determine what year+month combinations belong to this event --- 
    
    original_entry = district_data[(district_data['year']==year_original)&
                                   (district_data['month']==month_original)].index[0]
    
    
    adjust_these_entries = get_entries_for_event(month_original, year_original, margin_of_error_months)
    
    # --- adjust appropriate entries ---- 
    for (m,y) in adjust_these_entries:
        
        try:
            row_in_dataset =  district_data[(district_data['year']==y)&(district_data['month']==m)].index[0]
        except:
            # gave errors for events that occured at our final time point in the dataset: 
            row_in_dataset = district_data.index[-1]
      
        DroughtData['drought_reported'].loc[row_in_dataset] = True
        
        DroughtData['drought_desinventar'].loc[row_in_dataset] = DroughtData['drought_desinventar'].loc[original_entry]
        DroughtData['drought_news_article'].loc[row_in_dataset] = DroughtData['drought_news_article'].loc[original_entry]

        
    
#     if counter == 3:
#         break 
#     counter += 1
    
# print(district_data['District'].iloc[0], month_original, year_original)
# print('desinventar: ' , DroughtData['drought_desinventar'].loc[original_entry])
# print('news article: ' , DroughtData['drought_news_article'].loc[original_entry])

# Save to a .csv file 

In [16]:
DroughtData.to_csv('Droughts_satelite_and_events_1month.csv', index=False)

In [53]:
DroughtData[DroughtData.drought_reported][['month','year','date']]

,month,year,date
22,1,2002,2002-01-01
41,8,2003,2003-08-01
106,1,2009,2009-01-01
111,6,2009,2009-06-01
133,4,2011,2011-04-01
...,...,...,...
38938,3,2009,2009-03-01
38943,8,2009,2009-08-01
38962,3,2011,2011-03-01
39177,8,2009,2009-08-01


## keep track of districts with a reported droughts 

In [54]:
df = pd.DataFrame()
districts = []
drought_any = []

event_desinventar = []
event_news = []

for district, group in DroughtData.groupby('District'):
    if  sum(group['drought_reported']) > 0: 
        drought_any.append(True)
        

            
        
        if (sum(group['drought_desinventar']) >0) and (sum(group['drought_news_article']) >0):
            event_desinventar.append(True)
            event_news.append(True)
        
        elif sum(group['drought_desinventar']) >0:
            event_desinventar.append(True)
            event_news.append(False)
        elif sum(group['drought_news_article']) >0:
            event_news.append(True)
            event_desinventar.append(False)
        else:
            print("Weird...!")
        
    else: 
        drought_any.append(False)
        event_desinventar.append(False)
        event_news.append(False)
        
    districts.append(district)
    
    
df['District'] = districts
df['drought_in_district'] = drought_any  
df['desinventar'] = event_desinventar
df['news_articles'] = event_news

In [55]:
df.to_csv('districts_with_droughts_Uganda_Kenya.csv', index=False)

In [56]:
np.sum( df['desinventar'] & df['news_articles']     )

29

In [57]:
df

,District,drought_in_district,desinventar,news_articles
0,ABIM,True,True,True
1,ADJUMANI,True,False,True
2,AGAGO,False,False,False
3,ALEBTONG,True,False,True
4,AMOLATAR,True,True,False
...,...,...,...,...
164,WAKISO,True,False,True
165,Wajir,True,True,False
166,West Pokot,True,True,False
167,YUMBE,True,True,False
